In [ ]:
import numpy as np
import matplotlib.pyplot as plt

## Exploring Class Inheritance using Neuron Models

To explore class inheritance, we will be creating sub-classes of various neuron types using a base class called <code>Neuron</code>.

<code>Neuron</code> is defined below. In addition to <code>self</code>, this class takes 7 input parameters, but three parameters have been defined with default behavior in the <code>__init__()</code> method.

Running the code block below will define the object <code>neuron</code> from our base class, and the method <code>generate_spike_data()</code> will model neuron activity based on the parameters used to initialize <code>neuron</code>.

In [ ]:
#Define Base class for Izhikevich Neurons
class Neuron:
    def __init__(self, a, b, c, d, v0=-65, u0=None, dt=0.1):
        self.a = a
        self.b = b
        self.c = c
        self.d = d
        self.dt = dt
        
        self.v = v0
        self.u = u0 if u0 is not None else b*v0


    def reset(self):
        self.v = self.c
        self.u += self.d


    def time_step(self,I):
        dv = 0.04*self.v**2 + 5*self.v + 140 - self.u + I #Compute dv
        du = self.a * (self.b*self.v - self.u) #Compute du

        self.v += dv * self.dt #Update v 
        self.u += du * self.dt #Update u

        spike = False          #Check for spiking
        if self.v >= 30:
            spike = True
            self.reset()

        return self.v, self.u, spike


    def generate_spike_data(self, I, T):
        n_steps = int(T/self.dt)
        v_trace = np.zeros(n_steps)
        spike_times = []

        if np.isscalar(I):
            I = np.ones(n_steps) * I

        for t in range(n_steps):
            v, u, spike = self.time_step(I[t])
            v_trace[t] = v

            if spike:
                spike_times.append(t * self.dt)

        return v_trace, spike_times

<div class="alert alert-success">
    <b>Task:</b> Answer the following questions about the code: <br>
    1. What are the attributes of <code>Neuron</code>? <br>
    2. What are the methods of <code>Neuron</code>?
</div>

## The Izhikevich Neuron Model

The Izhikevich neuron model is a simplified version of the biologically motivated Hodgkin-Huxley model for describing the firing activity of neurons. Using the parameters <code>a</code>,<code>b</code>,<code>c</code>, and <code>d</code>, Izhikevich neurons are able to capture a diverse range of spiking behaviors seen in Hodgkin-Huxley models (and real neurons!) while being more computationally tractable.
The table below contains each parameter with its interpretation:

|Parameter |Description                                              |
|----------|---------------------------------------------------------|
|a         |Rate of recovery to baseline after spiking               |
|b         |Sensitivity to subthreshold membrane voltage flucutations|
|c         |Post-spike membrane reset voltage                        |
|d         |Post-spike recovery/ spike suppression                   |

In addition to these model parameters, the Izhikevich neuron model can be used to generate a voltage trace by using an input current <code>I</code> applied for <code>T</code> ms. The full model is given by the equations below:

<center>$\frac{dv}{dt} = 0.04v^{2}+5v+140-u+I$<br>
$\frac{du}{dt} = a(bv-u)$<br>
if $v \geq 30 mV$:<br>
    $v=c$<br>
    $u=u+d$</center>

Some typical values for each parameter are given in the code-block below. We'll use these parameters to generate data for a neuron to test that our <code>Neuron</code> base class is functioning properly. In addition to model parameters, we'll set the amplitude of the input current by setting <code>I=10</code>, and we'll set the duration of the input current (in ms) by setting <code>T=50</code>.

In [ ]:
#Test params: "Typical Values" from Izhikevich paper
a = 0.02
b = 0.2
c = -65
d = 2

I = 10 #mA
T = 50 #ms

Now that we've defined some parameters for our neuron, we can define our object <code>neuron</code> using input parameters <code>a</code>,<code>b</code>,<code>c</code>, and <code>d</code>.

In [ ]:
neuron = Neuron(...)

Next we want to generate data using our <code>neuron</code> object. To do that, use the method <code>generate_spike_data()</code>.

In [ ]:
data, spikes = ...

To plot our data accurately, we'll want to create a 1-D numpy array to keep track of all of our timepoints. First, we have to calculate the total number of timesteps in our data. This can be calculated with the formula: <br>
<code>num_steps = int(T/dt)</code> where <code>dt</code> is the duration of each timestep.

Once we've calculated <code>num_steps</code>, we can use <code>np.arange()</code> to create a 1-D array of timesteps. Then we can multiply this vector by <code>dt</code> to create our properly-scaled time array. Assign this array to the variable name <code>time</code>.


In [ ]:
time = ...

Run the code block below to plot the results!

In [ ]:
plt.plot(time, data, label='Basic Neuron')
plt.title("Voltage Trace of Basic Neuron")
plt.xlabel("Time (ms)")
plt.ylabel("Voltage (mV)")
plt.legend()

<div class="alert alert-success">
    <b>Task:</b> Generate a few neurons by altering the values for <code>a</code>, <code>b</code>, <code>c</code>, <code>d</code> in the code above, and re-plot the results. What changes when you change these parameters?
</div>

## Describing four types of Izhikevich Neurons

To create different sub-classes of neurons using our <code>Neuron</code> base class, we'll need to change our class header to include <code>Neuron</code> in parentheses like:

<code> class NeuronSubClass(Neuron):</code>

This will allow us to initialize our new subclass with the attributes and methods defined within <code>Neuron</code>. To define each of our neuron sub-classes, we want to store different values for <code>a</code>, <code>b</code>, <code>c</code>, <code>d</code> in the initialization.

Notice that <code>Neuron</code> also takes the arguments <code>v0=-65</code>,<code>u0=None</code>, and <code>dt=0.1</code>. We want these to be modifiable regardless of the neuron sub-class. To do this, we can pass in the <code>**kwargs</code> argument in our <code>__init__()</code> methods as below:

<code> def __init__(self, **kwargs):
        super().__init__(a=a_value, b=b_value, c=c_value, d=d_value, **kwargs) </code>

Define The following sub-classes using parameters in the table below:

|Sub-class Name        |a-value|b-value|c-value|d-value|
|----------------------|-------|-------|-------|-------|
|RegularSpiking        |0.02   |0.2    |-65    |8      |
|IntrinsicallyBursting |0.02   |0.2    |-55    |4      |
|Chattering            |0.02   |0.2    |-50    |2      |
|FastSpiking           |0.1    |0.2    |-65    |2      |

<div class="alert alert-success">
    <b>Task:</b> Create a subclass for each of the four neuron types described above by using different values for <code>a</code>, <code>b</code>, <code>c</code>, and <code>d</code>.
</div>

In [ ]:
class RegularSpiking(Neuron):

        
class IntrinsicallyBursting(Neuron):


class Chattering(Neuron):


class FastSpiking(Neuron):



<div class="alert alert-success">
    <b>Task:</b> Create a dictionary that maps neuron types to the appropriate class, and loop over the dictionary to plot each voltage trace on the same plot. Include a legend using the dictionary keys as data labels. 
</div>

In [ ]:
plt.figure(figsize=(10,8))
 

#Create a dictionary to store different neuron names
neurons = {'RS':...,
           'IB':...,
           'CH':...,
           'FS':...}

#Create a for loop that loops over keys and values in neurons. 
for name, neuron in neurons.items():
    v, spikes = ...
    #Plot the voltage trace using *name* as the label for your legend
    plt.plot(...)

plt.title("Izhikevich Neurons")
plt.xlabel("Time (ms)")
plt.ylabel("Voltage (mV)")
plt.legend()
plt.show()
    

Great! We can plot all of the voltage traces from each neuron! However, our plot looks busy, and even with the legend it's difficult to see the response of each neuron to stimulation. Let's fix this by plotting each voltage trace on separate axes. We can easily do this using <code>plt.subplots()</code>. To do this, take the following steps:

1. Generate spike data for each neuron sub-class
2. Using <code>time</code>, plot the time-course of your spiking data for each neuron on a separate axis. Title each plot with the sub-class name.
3. Title your figure "Izhikevich Neuron Subclasses" by calling <code>figure.suptitle()</code>
4. Call <code>figure.tight_layout()</code> to improve figure formatting

<div class="alert alert-success">
    <b>Task:</b> Using <code>plt.subplots()</code> make a figure where you plot each neuron trace on different axes. Give each subplot an informative title and a separate title for the whole figure.  
</div>

In [ ]:
#Generate spike data
rs_neuron = ...
rs_data, rs_spikes = ...

ib_neuron = ...
ib_data, ib_spikes = ...

ch_neuron = ...
ch_data, ch_spikes = ...

fs_neuron = ...
fs_data, fs_spikes = ...

In [ ]:
fig, ax = plt.subplots(2,2)
#Plot the voltage traces over time for each neuron

#Title your figure

#Fix the layout of your figure
